# Adatimport a választási adatok elemzéséhez

Adatok a valasztas.hu és a KSH adatai alapján. Letöltés: 2019. május.

Megjegyzés: a valasztas.hu oldalán 2019. május végén megtalálhatóak voltak a belföldi szavazókörökre lebontott eredmények letölthető, Excel formátumban, azonban 2019. június 3-án már nem.

In [1]:
%run definitions.ipynb

In [2]:
try:
    verbose_print == True
except NameError:
    verbose_print = True
    print("verbose_print = True")

verbose_print = True


### Változók

In [3]:
print("Levélszavazatok forrása: https://www.valasztas.hu/levelszavazas-eredmenye_ep2019")
levelszavazatok = {"FIDESZ": 55287,
                    "MOMENTUM": 553,
                    "MI HAZÁNK": 365,
                    "DK": 343,
                    "MKKP": 336,
                    "MSZP-PÁRBESZÉD": 258,
                    "LMP": 208,
                    "JOBBIK": 187,
                    "MUNKÁSPÁRT": 71,
                  }

levelszavazat_regisztralt =  115325
levelszavazat_beerkezett = 63762
levelszavazat_ervenyes = 57777
levelszavazat_ervenytelen = 5985
levelszavazat_ervenyes_szavazasi_iratokban_szavazolapok = 57774
levelszavazat_ervenyes_szavazasi_iratokban_ervenytelen_szavazolapok = 166
levelszavazat_ervenyes_szavazat = 57608


Levélszavazatok forrása: https://www.valasztas.hu/levelszavazas-eredmenye_ep2019


In [4]:
print("Külképviseleti adatok forrása: https://www.valasztas.hu/kulkepviseleti-szavazas-eredmenye_ep2019")
kulkepviseleti_szavazatok = {"FIDESZ":7222,
                            "MOMENTUM":5078,
                            "DK":1678,
                            "MSZP-PÁRBESZÉD":1113,
                            "MKKP":913,
                            "LMP":700,
                            "JOBBIK":400,
                            "MI HAZÁNK":362,
                            "MUNKÁSPÁRT":19,
                            }

kulkepvis_regisztralt = 20291
kulkepvis_megjelent = 17551
kulkepvis_nemszavazott = 2740
kulkepvis_beerkezett_ervenyes_boritek = 17551
kulkepvis_ervenytelen_boritek = 194
kulkepvis_ervenyes_boritek_belyegzonelkuli_szavazolap = 5
kulkepvis_ervenyes_boritek_lebelyegzett_szavazolap = 17546
kulkepvis_ervenytelen_lebelyegzett_szavazolap = 61
kulkepvis_ervenyes_szavazolap = 17485

Külképviseleti adatok forrása: https://www.valasztas.hu/kulkepviseleti-szavazas-eredmenye_ep2019


In [5]:
partok = ["MSZP-PÁRBESZÉD", "MKKP", "JOBBIK", "FIDESZ", "MOMENTUM", "DK", "MI HAZÁNK", "MUNKÁSPÁRT", "LMP"]
ellenzeki_partok = [p for p in partok if p != "FIDESZ"]

In [6]:
partok_szine = ['#c30020', #MSZP
                '#333333', #MKKP
                '#178429', #Jobbik
                '#fd8100', #Fidesz
                '#8e6fcd', #Momentum
                '#00adef', #DK
                'grey', #Mi Hazánk -- nincs színkód
                'darkred', #Munkáspárt
                '#73c92d', #LMP
                ]

ellenzeki_partok_szine = [partok_szine[i] for i in range(len(partok)) if i != partok.index("FIDESZ")]

## Adatok beimportálása

In [7]:
print("Beföldi választási adatok forrása (2019. május): https://www.valasztas.hu/ep2019")
print("Települési adatok forrása: KSH 2011-es helyiségnévkönyv: http://www.ksh.hu/docs/hun/hnk/Helysegnevkonyv_adattar_2011.xls")
print("Megyei szintű adatok forrása: KSH: https://www.ksh.hu/")

Beföldi választási adatok forrása (2019. május): https://www.valasztas.hu/ep2019
Települési adatok forrása: KSH 2011-es helyiségnévkönyv: http://www.ksh.hu/docs/hun/hnk/Helysegnevkonyv_adattar_2011.xls
Megyei szintű adatok forrása: KSH: https://www.ksh.hu/


In [8]:
valasztasi_adatok = pd.read_csv("dataout/valasztasi_eredmenyek.csv")
megyei_adatok = pd.read_csv("dataout/megyei_adatok.csv")
helyisegnevkonyv = pd.read_csv("dataout/helyisegnevkonyv.csv")

In [9]:
if verbose_print:   
    print("Balatonakarattya (2014-ben lett önálló) a 2011-es helyiségnevek azonosak a választási adatokban szereplőkkel:",
     set(valasztasi_adatok["Település"].unique()) == (set(helyisegnevkonyv["község_neve_2011"].unique()) | {"Balatonakarattya"})
     )

Balatonakarattya (2014-ben lett önálló) a 2011-es helyiségnevek azonosak a választási adatokban szereplőkkel: True


In [10]:
if verbose_print:
    print(np.shape(valasztasi_adatok))
valasztasi_adatok = valasztasi_adatok.merge(helyisegnevkonyv, left_on="Település", right_on="község_neve_2011", how='outer')
if verbose_print:
    print(np.shape(valasztasi_adatok))

(3177, 19)
(3177, 41)


In [11]:
if verbose_print:
    print(np.shape(valasztasi_adatok))
valasztasi_adatok = valasztasi_adatok.merge(megyei_adatok, left_on="Megye", right_on="megye", how='outer')
if verbose_print:
    print(np.shape(valasztasi_adatok))

(3177, 41)
(3177, 235)


In [12]:
if verbose_print:
    print("A 2011-es megyék azonosak a választási adatokban szereplőkkel:",
     (valasztasi_adatok[valasztasi_adatok["Település"] != "Balatonakarattya"]["Megye"] == valasztasi_adatok[valasztasi_adatok["Település"] != "Balatonakarattya"]["megye_2011"]).all()
     )

A 2011-es megyék azonosak a választási adatokban szereplőkkel: False


In [13]:
if verbose_print:
    tmp = (valasztasi_adatok[valasztasi_adatok["Település"] != "Balatonakarattya"]["Megye"] == valasztasi_adatok[valasztasi_adatok["Település"] != "Balatonakarattya"]["megye_2011"])
    display(valasztasi_adatok.iloc[tmp[tmp == False].index.tolist(), :].loc[:, ["Település", "Megye", "község_neve_2011", "megye", "megye_2011"]])
    del tmp

,Település,Megye,község_neve_2011,megye,megye_2011
1922,Balatonvilágos,Somogy,Balatonvilágos,Somogy,Veszprém


Balatonvilágos a Wikipédia szerint "2013. január 1-jétől Somogy megyéhez tartozik", előtte Veszprém megyéhez.

In [14]:
del valasztasi_adatok["község_neve_2011"]
del valasztasi_adatok["megye"]
del valasztasi_adatok["megye_2011"]

In [15]:
np.shape(valasztasi_adatok)

(3177, 232)

In [16]:
valasztasi_adatok.iloc[:, :20].head(1)

,Település,Megye,Település típusa,Szavazókör db,Névjegyzékben szereplő választópolgárok száma,Megjelentek száma,Bélyegző nélküli szavazólapok száma,Lebélyegzett szavazólapok száma,"Érvénytelen, lebélyegzett szavazólapok száma",Érvényes szavazólapok száma,MSZP-PÁRBESZÉD,MKKP,JOBBIK,FIDESZ,MOMENTUM,DK,MI HAZÁNK,MUNKÁSPÁRT,LMP,község_jogállása_2011
0,Budapest 01,Budapest,Fővárosi kerület,20,20315,12623,7,12610,45,12565,1035,508,241,5696,2631,1730,200,31,493,fővárosi kerület


**Helyiségek státusának változása 2011 óta**

In [17]:
if verbose_print:
    print(valasztasi_adatok["Település típusa"].unique())
    print(valasztasi_adatok["község_jogállása_2011"].unique())

['Fővárosi kerület' 'Község' 'Város' 'Megyei jogú város']
['fővárosi kerület' 'község' 'nagyközség' 'város'
 'megyeszékhely, megyei jogú város' 'megyei jogú város' nan]


In [18]:
if verbose_print:
    tmp = valasztasi_adatok[ (valasztasi_adatok["Település típusa"] == 'Község') & ( (valasztasi_adatok["község_jogállása_2011"] != "nagyközség") & (valasztasi_adatok["község_jogállása_2011"] != "község") ) ]
    print("\nKözség 2019-ben, de nem község vagy nagyközség 2011-ben:", len(tmp))
    print("\t", tmp["Település"].tolist())

    tmp = valasztasi_adatok[ (valasztasi_adatok["Település típusa"] == 'Város') & (valasztasi_adatok["község_jogállása_2011"] != "város") ]
    print("\nVáros 2019-ben, de nem 2011-ben:", len(tmp))
    print("\t", tmp["Település"].tolist())

    tmp = valasztasi_adatok[ (valasztasi_adatok["Település típusa"] == 'Megyei jogú város') & ( (valasztasi_adatok["község_jogállása_2011"] != 'megyeszékhely, megyei jogú város') & (valasztasi_adatok["község_jogállása_2011"] != 'megyei jogú város') ) ]
    print("\nMegyei jogú város 2019-ben, de nem megyei jogú város 2011-ben:", len(tmp))
    print("\t", tmp["Település"].tolist())


Község 2019-ben, de nem község vagy nagyközség 2011-ben: 1
	 ['Balatonakarattya']

Város 2019-ben, de nem 2011-ben: 18
	 ['Kondoros', 'Onga', 'Aba', 'Csákvár', 'Lébény', 'Gyöngyöspata', 'Verpelét', 'Besenyszög', 'Fegyvernek', 'Tát', 'Diósd', 'Kerepes', 'Őrbottyán', 'Piliscsaba', 'Sülysáp', 'Újhartyán', 'Ajak', 'Jánosháza']

Megyei jogú város 2019-ben, de nem megyei jogú város 2011-ben: 0
	 []


In [19]:
del valasztasi_adatok["község_jogállása_2011"]

**Szavazti arányok hozzáadása**

In [20]:
valasztasi_adatok["részvételi arány"] = valasztasi_adatok['Megjelentek száma'] / valasztasi_adatok["Névjegyzékben szereplő választópolgárok száma"]

valasztasi_adatok["érvényes szavazatok aránya"] = valasztasi_adatok["Érvényes szavazólapok száma"] / valasztasi_adatok["Megjelentek száma"]

for party in partok:
    valasztasi_adatok[party+"_%"] = valasztasi_adatok[party] / valasztasi_adatok["Érvényes szavazólapok száma"] * 100
    
    valasztasi_adatok[party+"_szavazóképes_lakosság_%"] = valasztasi_adatok[party] / valasztasi_adatok["Érvényes szavazólapok száma"] * 100
    

## Elemzéshez változók

In [21]:
cols_partra_adott_szavazatok_szazalekban = [c for c in valasztasi_adatok.columns if c.endswith("_%") and "szavazóképes" not in c]

In [22]:
valasztasi_adatok["Település típusa"] = pd.Categorical(valasztasi_adatok["Település típusa"],
                      categories=["Fővárosi kerület","Megyei jogú város","Város", "Község", 
                                  "BELFÖLDI", "Levélszavazat", "Külképviselet", "ÖSSZESÍTETT"],
                      ordered=True)

In [23]:
valasztasi_corr = valasztasi_adatok.loc[:, cols_partra_adott_szavazatok_szazalekban].corr()

### Megyei eredményekhez térkép

Shapefile forrás: https://gadm.org/download_country_v3.html

Mivel a file szabadon felhasználható, de nem terjeszthető, a térképfájlokat a fenti oldalról letölti az alábbi szkript, ha nem találja a `gadm36_HUN_1.shp` file-t a `datamap` könyvtárban.

In [24]:
map_file = "datamap/gadm36_HUN_1.shp"
if not os.path.exists(map_file): 
    if verbose_print:
        print("datamap/gadm36_HUN_1.shp file még nem létezik, shapefile letöltése...")
    map_url = "https://biogeo.ucdavis.edu/data/gadm3.6/shp/gadm36_HUN_shp.zip"
    download = requests.get(map_url)
    zipfile.ZipFile(BytesIO(download.content)).extractall('datamap')
map_df = gpd.read_file(map_file)
print("Térkép shapefile forrás: https://gadm.org/download_country_v3.html")


Térkép shapefile forrás: https://gadm.org/download_country_v3.html


In [25]:
map_df.NAME_1

0                BÃ¡cs-Kiskun
1                     Baranya
2                     BÃ©kÃ©s
3      Borsod-AbaÃºj-ZemplÃ©n
4                    Budapest
5                   CsongrÃ¡d
6                      FejÃ©r
7           Gyor-Moson-Sopron
8                HajdÃº-Bihar
9                       Heves
10      JÃ¡sz-Nagykun-Szolnok
11         KomÃ¡rom-Esztergom
12                   NÃ³grÃ¡d
13                       Pest
14                     Somogy
15    Szabolcs-SzatmÃ¡r-Bereg
16                      Tolna
17                        Vas
18                  VeszprÃ©m
19                       Zala
Name: NAME_1, dtype: object

In [26]:
map_df["NAME_1"] =  ['Bács-Kiskun',
                     'Békés',
                     'Baranya',
                     'Borsod-Abaúj-Zemplén',
                     'Budapest',
                     'Csongrád',
                     'Fejér',
                     'Győr-Moson-Sopron',
                     'Hajdú-Bihar',
                     'Heves',
                     'Jász-Nagykun-Szolnok',
                     'Komárom-Esztergom',
                     'Nógrád',
                     'Pest',
                     'Somogy',
                     'Szabolcs-Szatmár-Bereg',
                     'Tolna',
                     'Vas',
                     'Veszprém',
                     'Zala']


In [27]:
map_df.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,HUN,Hungary,HUN.1_1,Bács-Kiskun,Bacs-Bodrog-Kiskun,None,Megye,County,None,HU.BK,"POLYGON ((19.51615524 46.15385056, 19.51568413..."
1,HUN,Hungary,HUN.2_1,Békés,None,None,Megye,County,None,HU.BA,"POLYGON ((18.07125473 45.77189255, 18.07113266..."
2,HUN,Hungary,HUN.3_1,Baranya,None,None,Megye,County,None,HU.BE,"POLYGON ((20.89214516 46.27170944, 20.89204216..."
3,HUN,Hungary,HUN.4_1,Borsod-Abaúj-Zemplén,None,None,Megye,County,None,HU.BZ,"POLYGON ((20.7680397 47.67522812, 20.76270866 ..."
4,HUN,Hungary,HUN.5_1,Budapest,Budapeste,None,Fovaros,Capital City,None,HU.BU,"POLYGON ((18.94665337 47.37084198, 18.94456673..."


In [28]:
megyei_adatok = (valasztasi_adatok.loc[:, ["Megye", "Érvényes szavazólapok száma"]+partok]).groupby("Megye").sum().reset_index()
megyei_adatok

,Megye,Érvényes szavazólapok száma,MSZP-PÁRBESZÉD,MKKP,JOBBIK,FIDESZ,MOMENTUM,DK,MI HAZÁNK,MUNKÁSPÁRT,LMP
0,Baranya,125013,8291,3610,7575,62663,11799,23036,3907,581,3551
1,Borsod-Abaúj-Zemplén,200962,14367,3654,23498,103789,11643,31688,8078,1182,3063
2,Budapest,686313,62019,26290,21797,282583,119088,135821,15921,2321,20473
3,Bács-Kiskun,164947,8378,3619,11637,97011,11989,22157,6046,646,3464
4,Békés,109074,6725,2337,9440,57479,7419,18264,4465,758,2187
5,Csongrád,144543,16664,4316,7777,71283,13685,20070,7159,604,2985
6,Fejér,147781,7840,3816,10396,79545,13519,23659,5450,635,2921
7,Győr-Moson-Sopron,165859,8124,3783,9598,98139,14001,22791,5403,583,3437
8,Hajdú-Bihar,155708,8622,3442,11122,89325,12424,22056,4943,738,3036
9,Heves,99577,5792,1874,10220,52403,7105,16054,3935,458,1736


In [29]:
for i in range ( len( partok )):
    megyei_adatok[cols_partra_adott_szavazatok_szazalekban[i]] = megyei_adatok[partok[i]] / megyei_adatok["Érvényes szavazólapok száma"] * 100

In [30]:
# megyei adatok
megyei_geoadatok = map_df.merge(megyei_adatok.loc[:, ["Megye"]+cols_partra_adott_szavazatok_szazalekban],
                            left_on = "NAME_1", right_on = "Megye", how='outer')
megyei_geoadatok.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry,Megye,MSZP-PÁRBESZÉD_%,MKKP_%,JOBBIK_%,FIDESZ_%,MOMENTUM_%,DK_%,MI HAZÁNK_%,MUNKÁSPÁRT_%,LMP_%
0,HUN,Hungary,HUN.1_1,Bács-Kiskun,Bacs-Bodrog-Kiskun,None,Megye,County,None,HU.BK,"POLYGON ((19.51615524 46.15385056, 19.51568413...",Bács-Kiskun,5.079207,2.194038,7.054993,58.813437,7.268395,13.432800,3.665420,0.391641,2.100069
1,HUN,Hungary,HUN.2_1,Békés,None,None,Megye,County,None,HU.BA,"POLYGON ((18.07125473 45.77189255, 18.07113266...",Békés,6.165539,2.142582,8.654675,52.697251,6.801804,16.744595,4.093551,0.694941,2.005061
2,HUN,Hungary,HUN.3_1,Baranya,None,None,Megye,County,None,HU.BE,"POLYGON ((20.89214516 46.27170944, 20.89204216...",Baranya,6.632110,2.887700,6.059370,50.125187,9.438218,18.426884,3.125275,0.464752,2.840505
3,HUN,Hungary,HUN.4_1,Borsod-Abaúj-Zemplén,None,None,Megye,County,None,HU.BZ,"POLYGON ((20.7680397 47.67522812, 20.76270866 ...",Borsod-Abaúj-Zemplén,7.149113,1.818254,11.692758,51.646082,5.793633,15.768155,4.019665,0.588171,1.524169
4,HUN,Hungary,HUN.5_1,Budapest,Budapeste,None,Fovaros,Capital City,None,HU.BU,"POLYGON ((18.94665337 47.37084198, 18.94456673...",Budapest,9.036547,3.830614,3.175956,41.174071,17.351850,19.789950,2.319787,0.338184,2.983041


In [31]:
def colormid(c):
    return c + (0.25 * (1 - c))

def megyei_adatok_plot(figure_title, axis_title='Érvényes szavazatok pártra leadott %-os aránya:\n{}', my_vmax=65, my_figsize = (13,20)):
    fig = plt.figure(figsize=my_figsize)
    fig.subplots_adjust(hspace=0.5)
    fig.suptitle(figure_title)

    for i in range(len(partok)):
        party = partok[i]
        ## set colormap to match party color
        partycolor = mpl.colors.to_rgb(partok_szine[i])
        cdict = {'red': ((0.0, 1.0, 1.0),
                     (0.5, colormid(partycolor[0]), colormid(partycolor[0])),
                     (1.0, partycolor[0], partycolor[0])),
             'green': ((0.0, 1.0, 1.0),
                       (0.5,  colormid(partycolor[1]),  colormid(partycolor[1])),
                       (1.0, partycolor[1], partycolor[1])),
             'blue': ((0.0, 1.0, 1.0),
                      (0.5,  colormid(partycolor[2]),  colormid(partycolor[2])),
                      (1.0, partycolor[2], partycolor[2])),}
        my_cmap = mpl.colors.LinearSegmentedColormap('my_colormap',cdict,256)

        ax = fig.add_subplot(int(np.ceil(len(partok)/2)),2,i+1)
        #fig, ax = plt.subplots((1, figsize=(10, 6))
        ax.axis('off')
        ax.set_title(axis_title.format(party),
                     fontdict={'fontsize': '10', 'fontweight' : '3'})
        #ax.annotate('Forrás: valasztas.hu (választási adatok), DIVA-GIS (térkép forma)',
        #            xy=(0.1, .08), 
        #            xycoords='figure fraction', 
        #            horizontalalignment='left', 
        #            verticalalignment='top', 
        #            fontsize=12, 
        #            color='#555555')

        # Create colorbar
        if my_vmax == 'partymax':
            vmax_to_use = megyei_geoadatok[party+"_%"].max()
            vmin_to_use = megyei_geoadatok[party+"_%"].min()
        else: 
            vmax_to_use = my_vmax
            vmin_to_use = 0
        colorm = plt.cm.ScalarMappable(cmap=my_cmap, norm=plt.Normalize(vmin=vmin_to_use, vmax=vmax_to_use))
        # empty array for the data range
        colorm._A = []
        # add the colorbar to the figure
        cbar = fig.colorbar(colorm)

        megyei_geoadatok.plot(column=party+"_%", 
                              #cmap='Blues', 
                              #norm=plt.Normalize(vmin=0, vmax=100),
                              cmap=my_cmap, 
                              vmin=vmin_to_use, vmax=vmax_to_use,
                              linewidth=0.8, 
                              ax=ax, edgecolor='0.8')

In [32]:
megyei_adatok = (valasztasi_adatok.loc[:, ["Megye", 'Névjegyzékben szereplő választópolgárok száma', 'Megjelentek száma', 'Érvényes szavazólapok száma']]).groupby("Megye").sum().reset_index()
megyei_adatok["Részvételi arány %"] = megyei_adatok['Megjelentek száma'] / megyei_adatok['Névjegyzékben szereplő választópolgárok száma'] * 100
megyei_adatok["Érvényes szavazatok %"] = megyei_adatok['Érvényes szavazólapok száma'] / megyei_adatok['Megjelentek száma'] * 100

In [33]:
megyei_geoadatok = megyei_geoadatok.merge(megyei_adatok,
                            left_on = "NAME_1", right_on = "Megye", how='outer')

### Település típusa szerinti elemzéshez

In [34]:
def teljes_part_eredmeny(party):
    return (valasztasi_adatok[party].sum() + \
            levelszavazatok[party] + \
            kulkepviseleti_szavazatok[party]) / \
            (valasztasi_adatok["Érvényes szavazólapok száma"].sum() + levelszavazat_ervenyes + kulkepvis_ervenyes_szavazolap) * 100


In [35]:
telepulestipus_szerint = valasztasi_adatok.groupby("Település típusa").mean().loc[:, cols_partra_adott_szavazatok_szazalekban]
telepulestipus_szerint.loc["BELFÖLDI", :] = [valasztasi_adatok[party].sum() / valasztasi_adatok["Érvényes szavazólapok száma"].sum() * 100 for party in partok]
telepulestipus_szerint.loc["Levélszavazat", :] = [levelszavazatok[party] / levelszavazat_ervenyes_szavazat * 100 for party in partok]
telepulestipus_szerint.loc["Külképviselet", :] = [kulkepviseleti_szavazatok[party] / kulkepvis_ervenyes_szavazolap * 100 for party in partok]
telepulestipus_szerint.loc["ÖSSZESÍTETT", :] = pd.Series([teljes_part_eredmeny(party) for party in partok], index=list(map(lambda x: x+"_%", partok)))

In [36]:
telepulestipus_szerint

,MSZP-PÁRBESZÉD_%,MKKP_%,JOBBIK_%,FIDESZ_%,MOMENTUM_%,DK_%,MI HAZÁNK_%,MUNKÁSPÁRT_%,LMP_%
Település típusa,,,,,,,,,
Fővárosi kerület,8.792431,3.867999,3.203240,41.628195,17.348330,19.417060,2.350369,0.344670,3.047706
Megyei jogú város,7.754528,2.656323,6.236065,48.073557,10.265670,18.838494,3.357633,0.416314,2.401415
Város,5.791364,2.109993,8.090840,55.780774,6.961927,15.265971,3.778023,0.466506,1.754602
Község,4.545690,1.755409,8.538844,63.585713,4.511150,11.351873,3.618691,0.532731,1.559898
BELFÖLDI,6.720124,2.640663,6.467346,51.884112,9.980377,16.347060,3.340595,0.422975,2.196748
Levélszavazat,0.447854,0.583252,0.324608,95.971046,0.959936,0.595403,0.633593,0.123247,0.361061
Külképviselet,6.365456,5.221619,2.287675,41.303975,29.042036,9.596797,2.070346,0.108665,4.003432
ÖSSZESÍTETT,6.613902,2.619388,6.344016,52.560049,9.926197,16.050808,3.289102,0.416396,2.175274


In [37]:
telepulestipus_szerint_osszes_telepules = pd.melt(valasztasi_adatok.loc[:, ["Település típusa"]+cols_partra_adott_szavazatok_szazalekban], id_vars="Település típusa")
telepulestipus_szerint_osszes_telepules.sort_values(by='Település típusa', inplace=True)
telepulestipus_szerint_osszes_telepules.columns = ['Település típusa', 'Párt', 'Érvényes szavazatok pártra leadott %-os aránya']
telepulestipus_szerint_osszes_telepules["Párt"] = telepulestipus_szerint_osszes_telepules["Párt"].map(lambda x: x[:-2])
telepulestipus_szerint_osszes_telepules["Párt"] = pd.Categorical(telepulestipus_szerint_osszes_telepules["Párt"],
                             #categories=list(map(lambda x: x+"_%", partok)),
                             categories = partok,
                             ordered=True)
telepulestipus_szerint_osszes_telepules['Település típusa'] = telepulestipus_szerint_osszes_telepules['Település típusa'].cat.remove_unused_categories()

In [38]:
if verbose_print:
    display(valasztasi_adatok.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3177 entries, 0 to 3176
Columns: 251 entries, Település to LMP_szavazóképes_lakosság_%
dtypes: category(1), float64(147), int64(100), object(3)
memory usage: 6.1+ MB


### Korrelációk az egyes (többnyire megyei) mutatókkal

Fontos hangsúlyozni, hogy a KSH területi adatai többnyire megye-szintűek, így a legtöbb rendelkezésre álló mutató az adott település megyéjét jellemzi, nem magát a települést.

A **2011-es adatok település-szintűek**, a 2011-es helyiségnévkönyvből származnak. Az évszám nélküli mutatók mind a választással kapcsolatos, település szintű mutatók. (Egy fővárosi kerület egy településnek minősül.)

A legtöbb **megyei** mutatóban egy **2001-2005** közötti évből, valamint **2017-ből vagy 2018-ból** származnak az adatok (attól függően, hogy mely években voltak a teljes országot lefedő adatok), emellett a mutatókra a későbbi és a korábbi év arányát is használjuk mutatóként.

In [39]:
telepulesek_szama = len(valasztasi_adatok)
kozsegek_szama = len(valasztasi_adatok[valasztasi_adatok["Település típusa"] == "Község"])
nempest_kozsegek_szama = len(valasztasi_adatok[(valasztasi_adatok["Település típusa"] == "Község") & (valasztasi_adatok["Megye"] != "Pest")])
varosok_szama = len(valasztasi_adatok[valasztasi_adatok["Település típusa"] == "Város"])
nempest_varosok_szama = len(valasztasi_adatok[(valasztasi_adatok["Település típusa"] == "Város") & (valasztasi_adatok["Megye"] != "Pest")])
print("\nÖsszes település (egy fővárosi kerület egy településnek minősül):", telepulesek_szama)
print("\nA településekből község:", 
      kozsegek_szama,
     "({}%)".format(round(kozsegek_szama/telepulesek_szama*100, 2) ))
print("\tA községekből nem Pest megyei község:", 
      nempest_kozsegek_szama,
     "({}%)".format(round(nempest_kozsegek_szama/kozsegek_szama*100, 2) ))
print("\nA településekből (nem megyei/fő-) város:", 
      varosok_szama,
     "({}%)".format(round(varosok_szama/telepulesek_szama*100, 2) ))
print("\tA városokból nem Pest megyei város:", 
      nempest_varosok_szama,
     "({}%)".format(round(nempest_varosok_szama/varosok_szama*100, 2) ))


Összes település (egy fővárosi kerület egy településnek minősül): 3177

A településekből község: 2809 (88.42%)
	A községekből nem Pest megyei község: 2676 (95.27%)

A településekből (nem megyei/fő-) város: 322 (10.14%)
	A városokból nem Pest megyei város: 269 (83.54%)


In [40]:
input_vars = valasztasi_adatok.loc[:, [c for c in valasztasi_adatok.columns if all([party not in c for party in partok]) ]].copy()

In [41]:
if verbose_print:
    print("Nemnumerikus bemeneti oszlopok:",
    input_vars.select_dtypes(exclude=['number']).columns.tolist())

Nemnumerikus bemeneti oszlopok: ['Település', 'Megye', 'Település típusa', 'kistérség_székhelye_2011']


In [42]:
for c in ["Megye", "Település típusa"]:
    input_vars = pd.concat([input_vars, pd.get_dummies(input_vars[c])], axis=1)
    del input_vars[c]

In [43]:
if False:
    input_vars = input_vars.select_dtypes(include=['number'])
    for c in input_vars.columns:
        if len(input_vars[input_vars[c].isna()]) > 0:
            print(c, len(input_vars[input_vars[c].isna()]))
            input_vars[c].fillna(0, inplace=True)

**Korrelációk az egyes mutatók és a pártok eredményei között -- az összes belföldi településre**

In [44]:
result = pd.DataFrame()
for party in partok:
    target = valasztasi_adatok[party+"_%"]
    result_party = input_vars.corrwith(target)
    result_party.name = party
    result = pd.concat([result, result_party], axis=1, sort=False)
    
result_above20_r = result[(result.abs() > 0.2).any(1) ]

#### Korrelációk az egyes mutatók és a pártok eredményei között -- csak a városokra szorítkozva

In [45]:
result_varos = pd.DataFrame()
for party in partok:
    target = valasztasi_adatok[valasztasi_adatok["Település típusa"] == "Város"][party+"_%"]
    result_party = input_vars[valasztasi_adatok["Település típusa"] == "Város"].corrwith(target)
    result_party.name = party
    result_varos = pd.concat([result_varos, result_party], axis=1, sort=False)

result_varos_above20_r = result_varos[(result_varos.abs() > 0.2).any(1) ]

#### Korrelációk az egyes mutatók és a pártok eredményei között -- csak a községekre szorítkozva

Mivel a főváros, sőt a városok és a községek közti gazdasági és egyéb különbség jelentős, és azt már láttuk, hogy a település típusa fontos a pártok eredménye szempontjából.

In [46]:
result_kozseg = pd.DataFrame()
for party in partok:
    target = valasztasi_adatok[valasztasi_adatok["Település típusa"] == "Község"][party+"_%"]
    result_party = input_vars[valasztasi_adatok["Település típusa"] == "Község"].corrwith(target)
    result_party.name = party
    result_kozseg = pd.concat([result_kozseg, result_party], axis=1, sort=False)
    
result_kozseg_above20_r = result_kozseg[(result_kozseg.abs() > 0.2).any(1) ]

#### Pest megyén kívüli községek esetén a korrelációk egyes mutatókkal

In [47]:
mask = (valasztasi_adatok["Település típusa"] == "Község") & (valasztasi_adatok["Megye"] != "Pest")
result_kozseg_nonpest = pd.DataFrame()
for party in partok:
    target = valasztasi_adatok[ mask ][party+"_%"]
    result_party = input_vars[ mask ].corrwith(target)
    result_party.name = party
    result_kozseg_nonpest = pd.concat([result_kozseg_nonpest, result_party], axis=1, sort=False)
result_kozseg_nonpest_above20_r = result_kozseg_nonpest[(result_kozseg_nonpest.abs() > 0.2).any(1) ]

In [48]:
corr_vars_of_interest = ["élettartam_ffi_2017",
                        "élettartam_nő_2017",
                        "életkorátlag_ffi_arány_2018_2001",
                        "életkorátlag_nő_arány_2018_2001",
                        "nettókereset_2017",
                        "aktivitási_arány_2018",
                        "aktivitási_arány_arány_2018_2006",
                        "gdp_per_fo_ezerft_2017",
                        "gdp_orszagos_atlag_szazaleka_2017",
                        "vendégéjszakák_2018",
                        "vendégéjszakák_külföldi_2018",
                        "vendégéjszakák_arány_2018_2001",
                        "személygépkocsik_2018",
                        "internet_összes_vezetékes_2017",
                        "gyógyszertárak_2018",
                        "nonprofit_összesen_2017",
                        "nonprofit_összesen_arány_2017_2005",
                        "bűnelkövetők_arány_fiatalkorú_összes_2017",
                        "bűnelkövetők_arány_összes_fiatalkorú_arány_2017_2001",
                        "bűncselekmények_2017",
                        "bűncselekmények_arány_2017_2001",
                        "felsőfokúnappalihallgatók_2017",
                        "felsőfokúnappalihallgatók_arány_2017_2001",
                        "pedagógusok_óvodai_2017",
                        "köznevelési_intézmények_óvodai_arány_2017_2001",
                        "szociális_étkeztetésben_részesülők_2018",
                        "hátrányoshelyzetű_kiskorúak_2017",
                        "orvosok_száma_arány_2017_2002",
                        "háztartás_áramfogyasztás_kwh_2017",
                        "közműolló_csatornázott_lakások_vízközművezettek_százalékában_2017",
                        "részvételi arány"]


### Nemzetiségi önkormányzatok

In [49]:
nemzetisegi_onkorm = [c for c in valasztasi_adatok.columns if c.startswith("önkormányzat_")]
len(nemzetisegi_onkorm)

13

### Belföldi összesített eredmények

In [50]:
belfoldi_osszesitett = (valasztasi_adatok.loc[:, ["Érvényes szavazólapok száma"]+partok]).sum()
for i in range ( len( partok )):
    belfoldi_osszesitett[cols_partra_adott_szavazatok_szazalekban[i]] = belfoldi_osszesitett[partok[i]] / belfoldi_osszesitett["Érvényes szavazólapok száma"] * 100
#belfoldi_osszesitett.loc[["Érvényes szavazólapok száma"]+partok]